# ERROR: Target did not ack

This is one of the errors that users most often have trouble resolving (if we go by [forum questions](https://forum.newae.com/search?q=target%20did%20not%20ack)).

The actual error is something like this:\
`(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack`\
(the line number will change as the codebase evolves)

Here we go over the most likely causes, and how to resolve them.

**Supported Capture Hardware:**\
✅ CW-Nano\
✅ CW-Lite\
✅ CW-Pro\
✅ CW-Husky

**Required ChipWhisperer software installation:**\
❌ not released yet\
✅ [develop branch](https://github.com/newaetech/chipwhisperer/tree/develop)

In [1]:
%run 'connect.ipynb'
cw.scope_logger.setLevel(cw.logging.WARNING) # turn warnings back on

In [2]:
target._name

'Simple Serial'

Let's confirm the default settings that [`connect.ipynb`](connect.ipynb) gave us via `scope.default_setup()` and `cw.target()`:

In [3]:
CLOCK = scope.clock.clkgen_freq
BAUD = target.baud
print('Clock: %f MHz\nBaud: %d bps' % (CLOCK, BAUD))

Clock: 7370129.870130 MHz
Baud: 38400 bps


We can confirm that communication is working by running a capture.

If your target is running `simpleserial-aes` firmware, this will run the target AES operation. For different target firmware, substitute with what's needed to make your target "go":

In [4]:
target.baud

38400

In [5]:
target.baud = 50123

In [6]:
target.baud = 38400

In [7]:
scope.io.tio1 = "serial_tx"
scope.io.tio2 = "serial_rx"

In [8]:
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"

In [9]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

USBErrorIO: LIBUSB_ERROR_IO [-1]

In [20]:
target.set_key(bytearray(16), always_send=True)

(ChipWhisperer Target ERROR|File SimpleSerial.py:318) Target did not ack


Warning: Device failed to ack

In [17]:
cw.capture_trace?

Signature:
cw.capture_trace(
    scope: Union[chipwhisperer.capture.scopes.OpenADC.OpenADC, chipwhisperer.capture.scopes.cwnano.CWNano],
    target: Union[chipwhisperer.capture.targets.CW305.CW305, chipwhisperer.capture.targets.CW305_ECC.CW305_ECC, chipwhisperer.capture.targets.CW310.CW310, chipwhisperer.capture.targets.CW340.CW340, chipwhisperer.capture.targets.SimpleSerial.SimpleSerial, chipwhisperer.capture.targets.SimpleSerial2.SimpleSerial2, chipwhisperer.capture.targets.SimpleSerial2.SimpleSerial2_CDC],
    plaintext: chipwhisperer.common.utils.util.CWByteArray,
    key: Optional[chipwhisperer.common.utils.util.CWByteArray] = None,
    ack: bool = True,
    poll_done: bool = False,
    as_int: bool = False,
    always_send_key=False,
) -> Optional[chipwhisperer.common.traces.Trace]
Docstring:
Capture a trace, sending plaintext and key

Does all individual steps needed to capture a trace (arming the scope
sending the key/plaintext, getting the trace data back, etc.). Uses
target.o

In [ ]:
assert not scope.adc.errors, scope.adc.errors